In [1]:
# imports
from z3 import *
from utils import *

In [2]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 3, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance0 = {
    "m" : 2,
    "n" : 2,
    "l" : [15, 10],
    "s" : [3, 2],
    "distances" : [
        [0, 3, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

instance2 = {
    "m" : 2,
    "n" : 3,
    "l" : [15, 10],
    "s" : [10, 2, 4],
    "distances" : [
        [0, 3, 3, 1],
        [3, 0, 4, 1],
        [3, 4, 0, 1],
        [3, 2, 1, 0]
    ]
}

In [16]:
# ONLY BINARY VARIABLES CAN BE USED
def MultipleCouriersPlanning(instance):
    m = instance["m"] # courriers
    n = instance["n"] # packages
    l = instance["l"] # weigths
    s = instance["s"] # sizes of courriers
    distances = instance["distances"] # distances between packages

    solver = Solver()

    # courires => [0..m) => [0, m-1]
    ## e.g. with m_couriers = 7 => m € [0, 6]

    # packages => [0, n+1) => [0, n]
    ## e.g. with n_packages = 8 => n € [0, 8] where 8 is the starting/ending point

    # times (k) => [0, n+2) => [0, n+1] where n is the number of packages

    # Variables

    # To codify that courrier i deliver package j at time k
    v = [[[Bool(f"x_{i}_{j}_{k}") for k in range(n+2)] for j in range (n+1)] for i in range(m)]

    # To codify the fact that package j has weight x
    weigths_bin_array = [[Bool(f"wba_{j}_{x}") for x in range(s[j])] for j in range (n)] 

    # Initialization constraints
    # 1. Put to 1 every variable that represents a kg in weigths
    for j in range (n):
        for x in range(s[j]):
            solver.add(weigths_bin_array[j][x])
    
    # Constraints
    # 1. Each courrier can carry at most l[i] kg
    for i in range (m):
        weigth_set = []
        for j in range (n):
            for k in range(1,n+1):
                for x in range(s[j]):
                    weigth_set.append(And(v[i][j][k], weigths_bin_array[j][x]))
        solver.add(at_most_k_seq(weigth_set, l[i], f"courier_{i}_load"))


    # 2. Each courrier i starts and end at position j = n
    for i in range (m):
        solver.add(And(v[i][n][0], v[i][n][n+1]))
    
    # depot j = n
    # time: 0...n+1

    # 3. Each courier can't be in two places at the same time 
    for i in range(m):
        for k in range(n+2):
            solver.add(PbEq([(v[i][j][k],1) for j in range(n+1)],1))    # exactly_one_np([v[i][j][k] for j in range(n+1)], "amo_package")
    
    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(PbEq([(v[i][j][k],1) for k in range(1,n+1) for i in range(m)],1))    # exactly_one_np([v[i][j][k] for k in range(1,n+1) for i in range(m)], "exactly_once")
    
    # Symmetry breaking constraints
    # 5. once a courier arrive to depot (j = n+1), it can't depart from there
    for i in range (m):
        for k in range(1, n):
            solver.add(
                Implies(v[i][n][k], v[i][n][k+1])
            )
    
    # symmetry breaking for courriers with equal capacity


    print(solver.check())
    model = solver.model()
    for i in range(m):
        print()
        print(f"Courier {i}:")
        for k in range(n+2):
            for j in range(n+1):
                if model[v[i][j][k]]:
                    print(f"Time {k} Place {j} : {model[v[i][j][k]]}")

MultipleCouriersPlanning(instance1)


sat

Courier 0:
Time 0 Place 7 : True
Time 1 Place 5 : True
Time 2 Place 0 : True
Time 3 Place 3 : True
Time 4 Place 7 : True
Time 5 Place 7 : True
Time 6 Place 7 : True
Time 7 Place 7 : True
Time 8 Place 7 : True

Courier 1:
Time 0 Place 7 : True
Time 1 Place 6 : True
Time 2 Place 2 : True
Time 3 Place 7 : True
Time 4 Place 7 : True
Time 5 Place 7 : True
Time 6 Place 7 : True
Time 7 Place 7 : True
Time 8 Place 7 : True

Courier 2:
Time 0 Place 7 : True
Time 1 Place 4 : True
Time 2 Place 1 : True
Time 3 Place 7 : True
Time 4 Place 7 : True
Time 5 Place 7 : True
Time 6 Place 7 : True
Time 7 Place 7 : True
Time 8 Place 7 : True


In [ ]:
courier 0:

k = 0 -> posto n=2
k = 1 -> porta pacco 0
k = 2 -> ???
k = 3 -> posto n=2

courier 1:

k = 0 -> posto n=2
k = 1 -> ???
k = 2 -> porta pacco 1
k = 3 -> posto n=2


SyntaxError: invalid syntax (1780622645.py, line 1)